In [25]:
#Get metadata of the articles from PMC API
#encoding=utf8
import importlib
import os
import urllib.request
from xlrd import open_workbook
from Bio import Entrez
import xml.dom.minidom
import xml.etree.ElementTree as ET
import pandas
import openpyxl
import pandas as pd
import tarfile
Entrez.email = "lhoang2@illinois.edu"

#Read list of PMIDs from txt file
citations_path = ("/Users/phuong/Downloads/Citations/XLSX/")
filelist = os.listdir(citations_path)

for file in filelist:
    if file.endswith("21.xlsx"):
        filename = citations_path + file
        df = pd.read_excel(filename)
        PMCID_list = df['PMCID']
        count = 0
        for PMCID in PMCID_list:
            if (PMCID != "None"):
                count +=1
        print (count)
        
        new_folder_path = citations_path + file.rstrip(".xlsx")
        os.makedirs(new_folder_path)

        #get the link to physical files
        for PMCID in PMCID_list:
            link = "https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=" + PMCID
            request = urllib.request.Request(link)
            result = urllib.request.urlopen(request)
            tree = ET.parse(result)
            doc = tree.getroot()
            for child in doc:
                if child.tag == "records":
                    article_download_link = ""
                    for child_level_2 in child:
                        if child_level_2.tag == "record":
                            for child_level_3 in child_level_2:
                                if ("link" in child_level_3.tag):
                                    link_format = child_level_3.attrib["format"]
                                    if (link_format == "pdf"):
                                        article_download_link = child_level_3.attrib["href"]

                                    elif (article_download_link=="") and (link_format == "tgz"):
                                        article_download_link = child_level_3.attrib["href"]
                    print (article_download_link)  
                    
                    if (article_download_link.endswith("pdf")):                    
                        urllib.request.urlretrieve(article_download_link, 
                                                               new_folder_path + "/" + PMCID + ".pdf")
                    elif (article_download_link.endswith("tar.gz")):
                        urllib.request.urlretrieve(article_download_link, 
                        new_folder_path + "/" + PMCID + ".tar.gz")

                        tar = tarfile.open(new_folder_path + "/" + PMCID + ".tar.gz", "r:gz")
                        for member in tar.getmembers():
                            if (member.name.endswith("pdf")):
                                f = tar.extractfile(member)
                                content = f.read()
                                with open(new_folder_path + "/" + PMCID + '.pdf', 'wb') as file:
                                    file.write(content)

13
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/ec/da/PMC5689094.tar.gz
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/cb/06/PMC5749464.tar.gz
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/e8/e9/PMC5844284.tar.gz
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/d9/07/PMC5915455.tar.gz
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/18/81/PMC5960167.tar.gz
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/0b/f3/PMC5968501.tar.gz
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/0c/c4/PMC6106427.tar.gz
ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/59/e1/PMC6102244.tar.gz
